## Imports

In [1]:
import torch
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import matplotlib.pyplot as plt
import os
import warnings

warnings.simplefilter("ignore", category=UserWarning)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


## Environment

In [2]:
env = gym.make("Pendulum-v1", render_mode="human", g=9.81)

In [3]:
env.reset()

(array([ 0.6246238 , -0.7809258 , -0.12713262], dtype=float32), {})

In [4]:
env.action_space

Box(-2.0, 2.0, (1,), float32)

In [5]:
env.observation_space

Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)

In [6]:
env.render()

In [ ]:
episodes = 5
for i in range(1, episodes+1):
    obs = env.reset(seed=42)
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, truncated, info = env.step(action)
        score += reward
    print(f"Episode: {i} Score: {score}")

env.close()

## Train Model

In [8]:
env = DummyVecEnv([lambda: env])

In [10]:
log_path = os.path.join("Training", "logs")
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path, device=device)

Using mps device


In [11]:
model.learn(total_timesteps=100000)

Logging to Training/logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 28   |
|    iterations      | 1    |
|    time_elapsed    | 71   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 28          |
|    iterations           | 2           |
|    time_elapsed         | 145         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.003071351 |
|    clip_fraction        | 0.016       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.42       |
|    explained_variance   | -0.00441    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.29e+03    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00198    |
|    std                  | 0.999       |
|    value_loss           | 9e+03       |
---

Use "tensorboard --logdir logs/" in the Training terminal to get to tensorboard

## Test Model

In [18]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(-952.4752577295294, 286.337085999703)

In [19]:
episodes = 5

ep = []
points = []

for i in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward        

    ep.append(i)
    points.append(score)
    
    print(f"Episode: {i} Score: {score}")

env.close()

Episode: 1 Score: [-874.0836]
Episode: 2 Score: [-763.905]
Episode: 3 Score: [-778.63055]
Episode: 4 Score: [-1099.1671]
Episode: 5 Score: [-1275.1355]
